# 4D Ionization Fraction Table
For an ionization table indexed by density, redshift, metallicity and temperature.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# You might need a different backend, but make sure it's interactive
%matplotlib ipympl
import h5py
import ipywidgets as widgets

In [ ]:
# Takes the ionization state as 0, 1, 2 and converts to I, II, III
# H, 0 -> HI, etc.
dec2roman = "I II III IV V VI VII VIII IX X XI XII XIII XIV XV XVI XVII XVIII XIX XX XXI XXII XXIII XXIV XXV XXVI XXVII XXVIII XXIX XXX XXXI".split()

In [ ]:
filename = "/Users/s1857441/Library/CloudStorage/OneDrive-UniversityofEdinburgh/Undergraduate/Year 5/MPhys Project/data/hm2012_ss_Z_hr.h5"  # Example using custom data

filein = h5py.File(filename, "r")

element = 'H'  # an initial value
density_arr, redshift_arr, metallicity_arr, temperature_arr = filein[element].attrs.values()

# Create sliders/dropdowns and link the element selector to change the ionization state slider
element_selector = widgets.Dropdown(options=filein.keys(), value='H', description="Element")
redshift_slider = widgets.SelectionSlider(options=redshift_arr, value=0, description="Redshift")
ionization_state_slider = widgets.IntSlider(0, 0, 1, description="Ionization State")
metallicity_slider = widgets.SelectionSlider(options=metallicity_arr, value=0, description="log(Z/Zsol)", readout_format=".2f")

# Create the plot
extent = (density_arr[0], density_arr[-1], temperature_arr[0], temperature_arr[-1])
im = plt.imshow(
    filein[element][ionization_state_slider.value, :, redshift_slider.index, metallicity_slider.index, :].T,
    vmin=-9,
    vmax=0,
    origin="lower",
    extent=extent,
    interpolation='None'
    )

plt.ylabel("log Temperature")
plt.xlabel("log density")
plt.colorbar(im)
title = plt.title(f"Ionization fraction of {element_selector.value} {dec2roman[ionization_state_slider.value]}")

# Add interactivity
# We could use widgets.interact, but this way means we don't need to search for the value, and can instead take the index directly
# It _looks_ worse, but in theory should be slightly more perfomative

def set_data(change):
    im.set_data(
        filein[element_selector.value][ionization_state_slider.value, :, redshift_slider.index, metallicity_slider.index, :].T
    )

def set_title(change):
    title.set_text(f"Ionization fraction of {element_selector.value} {dec2roman[ionization_state_slider.value]}")

def on_element_change(change):
    ionization_state_slider.max = filein[element_selector.value].shape[0] - 1

element_selector.observe(on_element_change)
element_selector.observe(set_title)
element_selector.observe(set_data)

ionization_state_slider.observe(set_title)
ionization_state_slider.observe(set_data)

redshift_slider.observe(set_data)
metallicity_slider.observe(set_data)

display(element_selector,
        ionization_state_slider,
        metallicity_slider,
        redshift_slider)

plt.show()


In [ ]:
plt.close()
filein.close()